Initial Imports

In [2]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from h2o.estimators import H2ODeepLearningEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator

# Server Start

In [3]:
h2o.init(min_mem_size='2G')

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)
  Starting server from /home/markel/.local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpq6sy9f51
  JVM stdout: /tmp/tmpq6sy9f51/h2o_markel_started_from_python.out
  JVM stderr: /tmp/tmpq6sy9f51/h2o_markel_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,4 months and 29 days
H2O_cluster_name:,H2O_from_python_markel_nbxe7t
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.857 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


## Training of the initial basic model

In [4]:
def load_and_prepare_data(file_path):
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()
    df['Label_Binary'] = df['Label'].apply(lambda x: 'BENIGN' if x == 'BENIGN' else 'ATTACK')
    
    # Forzar los tipos de las columnas
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])  # Forzar 'Timestamp' como datetime
    df['Label_Binary'] = df['Label_Binary'].astype('category')  # Forzar Label_Binary como categoría

    h2o_frame = h2o.H2OFrame(df)

    # Asegurarse de que las columnas sean categóricas donde sea necesario
    h2o_frame['Label_Binary'] = h2o_frame['Label_Binary'].asfactor()

    return h2o_frame

In [5]:
mon_tues_array = [
    "CICIDS2017/Monday-WorkingHours.pcap_ISCX.csv",
    "CICIDS2017/Tuesday-WorkingHours.pcap_ISCX.csv"
]
mon_tues_frames = [load_and_prepare_data(file) for file in mon_tues_array]
mon_tues = mon_tues_frames[0].rbind(mon_tues_frames[1])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
wed = load_and_prepare_data("CICIDS2017/Wednesday-workingHours.pcap_ISCX.csv")


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
# Lista de predictores y respuesta
predictors = [
    "Destination Port", 
    "Protocol","Flow Duration", "Total Fwd Packets", 
    "Total Backward Packets", "Total Length of Fwd Packets", 
    "Total Length of Bwd Packets", "Fwd Packet Length Max", 
    "Fwd Packet Length Min", "Fwd Packet Length Mean", "Fwd Packet Length Std", 
    "Bwd Packet Length Max", "Bwd Packet Length Min", "Bwd Packet Length Mean", 
    "Bwd Packet Length Std", "Flow Bytes/s", "Flow Packets/s", "Flow IAT Mean", 
    "Flow IAT Std", "Flow IAT Max", "Flow IAT Min", "Fwd IAT Total", 
    "Fwd IAT Mean", "Fwd IAT Std", "Fwd IAT Max", "Fwd IAT Min", 
    "Bwd IAT Total", "Bwd IAT Mean", "Bwd IAT Std", "Bwd IAT Max", 
    "Bwd IAT Min", "Fwd PSH Flags", "Bwd PSH Flags", "Fwd URG Flags", 
    "Bwd URG Flags", "Fwd Header Length", "Bwd Header Length", 
    "Fwd Packets/s", "Bwd Packets/s", "Min Packet Length", "Max Packet Length", 
    "Packet Length Mean", "Packet Length Std", "Packet Length Variance", 
    "FIN Flag Count", "SYN Flag Count", "RST Flag Count", "PSH Flag Count", 
    "ACK Flag Count", "URG Flag Count", "CWE Flag Count", "ECE Flag Count", 
    "Down/Up Ratio", "Average Packet Size", "Avg Fwd Segment Size", 
    "Avg Bwd Segment Size", "Fwd Header Length", "Fwd Avg Bytes/Bulk", 
    "Fwd Avg Packets/Bulk", "Fwd Avg Bulk Rate", "Bwd Avg Bytes/Bulk", 
    "Bwd Avg Packets/Bulk", "Bwd Avg Bulk Rate", "Subflow Fwd Packets", 
    "Subflow Fwd Bytes", "Subflow Bwd Packets", "Subflow Bwd Bytes", 
    "Init_Win_bytes_forward", "Init_Win_bytes_backward", "act_data_pkt_fwd", 
    "min_seg_size_forward", "Active Mean", "Active Std", "Active Max", 
    "Active Min", "Idle Mean", "Idle Std", "Idle Max", "Idle Min"
]

response = "Label"
response_binary = "Label_Binary"

In [8]:
def train_automl(train, valid=None, max_runtime_secs=60):
    aml = H2OAutoML(max_runtime_secs=max_runtime_secs, seed=1234, verbosity="info", nfolds=0, keep_cross_validation_predictions=False,
                        )
    aml.train(x=predictors, y=response, training_frame=train, validation_frame=valid)
    return aml


In [9]:
def train_binary(train, valid=None, max_runtime_secs=60, checkpoint_model=None):
    #TODO:checkpoint_model logic
    aml = H2OAutoML(max_runtime_secs=max_runtime_secs, seed=1234, verbosity="info", nfolds=0, keep_cross_validation_predictions=False,
                        include_algos=['DeepLearning', 'DRF'])
    aml.train(x=predictors, y=response_binary, training_frame=train, validation_frame=valid)
    return aml

In [10]:
def model_metrics_evaluation(model_predictions, ground_truth, label='ATTACK'):
    accuracy = accuracy_score(ground_truth.as_data_frame(), model_predictions.as_data_frame())
    f1 = f1_score(ground_truth.as_data_frame(), model_predictions.as_data_frame(), pos_label=label)
    recall = recall_score(ground_truth.as_data_frame(), model_predictions.as_data_frame(), pos_label=label)
    precision = precision_score(ground_truth.as_data_frame(), model_predictions.as_data_frame(), pos_label=label)
    confusion = confusion_matrix(ground_truth.as_data_frame(), model_predictions.as_data_frame(), labels=['BENIGN', 'ATTACK'])
    return accuracy, f1, recall, precision, confusion

### Pipeline

In [11]:

#Binary clasification
aml = train_binary(mon_tues)
print(aml.leaderboard)

AutoML progress: |
13:24:44.710: Project: AutoML_1_20240812_132444
13:24:44.711: Cross-validation disabled by user: no fold column nor nfolds > 1.
13:24:45.585: Setting stopping tolerance adaptively based on the training frame: 0.00101231013475982
13:24:45.586: Build control seed: 1234
13:24:45.599: Since cross-validation is disabled, and validation frame(s) were not provided, automatically split the training data into training, validation frame(s) in the ratio 90/10/0.
13:24:50.462: training frame: Frame key: AutoML_1_20240812_132444_training_py_3_sid_92aa    cols: 86    rows: 878202  chunks: 120    size: 308850310  checksum: 2033068135242734500
13:24:50.497: validation frame: Frame key: AutoML_1_20240812_132444_validation_py_3_sid_92aa    cols: 86    rows: 97625  chunks: 120    size: 85803392  checksum: 7598027722460235240
13:24:50.497: leaderboard frame: Frame key: AutoML_1_20240812_132444_validation_py_3_sid_92aa    cols: 86    rows: 97625  chunks: 120    size: 85803392  checksum: 

In [27]:
model = aml.leader

predictions_wed = model.predict(wed)
accuracy, f1, recall, precision, confusion = model_metrics_evaluation(predictions_wed['predict'], wed['Label_Binary'], label='BENIGN')
print(f"Accuracy: {accuracy}")
print(f"F1: {f1}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print("Confusion Matrix:")
print(confusion)


drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/home/markel/.local/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/home/markel/.local/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn

Accuracy: 0.6352174019745837
F1: 0.7769210396214422
Recall: 0.9999681840597594
Precision: 0.6352302404109204
Confusion Matrix:
[[440017     14]
 [252672      0]]


In [13]:
model = aml.leader
#TODO: Pipeline in wich we predict hour by hour while monitoring the metrics and retrain if one of the metrics is below a threshold
wed_hour_1 = wed[wed['Timestamp'] < '2017-07-05 09:00:00']
wed_hour_2 = wed[(wed['Timestamp'] >= '2017-07-05 09:00:00') & (wed['Timestamp'] < '2017-07-05 10:00:00')]
wed_hour_3 = wed[(wed['Timestamp'] >= '2017-07-05 10:00:00') & (wed['Timestamp'] < '2017-07-05 11:00:00')]
wed_hour_4 = wed[(wed['Timestamp'] >= '2017-07-05 11:00:00') & (wed['Timestamp'] < '2017-07-05 12:00:00')]
wed_hour_5 = wed[(wed['Timestamp'] >= '2017-07-05 12:00:00') & (wed['Timestamp'] < '2017-07-05 13:00:00')]
wed_hour_6 = wed[(wed['Timestamp'] >= '2017-07-05 13:00:00') & (wed['Timestamp'] < '2017-07-05 14:00:00')]
wed_hour_7 = wed[(wed['Timestamp'] >= '2017-07-05 14:00:00') & (wed['Timestamp'] < '2017-07-05 15:00:00')]
wed_hour_8 = wed[(wed['Timestamp'] >= '2017-07-05 15:00:00') & (wed['Timestamp'] < '2017-07-05 16:00:00')]
wed_hour_9 = wed[(wed['Timestamp'] >= '2017-07-05 16:00:00') & (wed['Timestamp'] < '2017-07-05 17:00:00')]
wed_hour_10 = wed[(wed['Timestamp'] >= '2017-07-05 17:00:00') & (wed['Timestamp'] < '2017-07-05 18:00:00')]

wed_hours = [wed_hour_1, wed_hour_2, wed_hour_3, wed_hour_4, wed_hour_5, wed_hour_6, wed_hour_7, wed_hour_8, wed_hour_9, wed_hour_10]

for hour in wed_hours:
    predictions_hour = model.predict(hour)
    accuracy, f1, recall, precision, confusion = model_metrics_evaluation(predictions_hour['predict'], hour['Label_Binary'], label='BENIGN')
    print(f"Accuracy: {accuracy}")
    print(f"F1: {f1}")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print("Confusion Matrix:")
    print(confusion)
    if f1 < 0.9:
        aml = train_binary(mon_tues.rbind(hour))
        model = aml.leader
        print(aml.leaderboard)
        print("Model retrained")
        print("Model metrics after retraining")
        predictions_hour = model.predict(hour)
        accuracy, f1, recall, precision, confusion = model_metrics_evaluation(predictions_hour['predict'], hour['Label_Binary'], label='BENIGN')
        print(f"Accuracy: {accuracy}")
        print(f"F1: {f1}")
        print(f"Recall: {recall}")
        print(f"Precision: {precision}")
        print("Confusion Matrix:")
        print(confusion)
    print("---------------------------------------------------")


drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/home/markel/.local/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/home/markel/.local/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn

Accuracy: 0.6352780340203522
F1: 0.7769474849602619
Recall: 0.9999545486567992
Precision: 0.6352711023843727
Confusion Matrix:
[[440011     20]
 [252624     48]]
AutoML progress: |
13:27:15.456: Project: AutoML_2_20240812_132715
13:27:15.456: Cross-validation disabled by user: no fold column nor nfolds > 1.
13:27:16.863: Setting stopping tolerance adaptively based on the training frame: 0.001
13:27:16.863: Build control seed: 1234
13:27:16.863: Since cross-validation is disabled, and validation frame(s) were not provided, automatically split the training data into training, validation frame(s) in the ratio 90/10/0.
13:27:25.143: training frame: Frame key: AutoML_2_20240812_132715_training_py_11_sid_92aa    cols: 86    rows: 1501555  chunks: 201    size: 507729468  checksum: 8435892949753223592
13:27:25.241: validation frame: Frame key: AutoML_2_20240812_132715_validation_py_11_sid_92aa    cols: 86    rows: 166975  chunks: 201    size: 127335862  checksum: -5896341377750273624
13:27:25.

/home/markel/.local/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/home/markel/.local/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn

Accuracy: 0.9986574332722682
F1: 0.9989430977464117
Recall: 0.9987932668380183
Precision: 0.9990929736143979
Confusion Matrix:
[[439500    531]
 [   399 252273]]
---------------------------------------------------
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/home/markel/.local/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/home/markel/.local/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn

ValueError: unknown is not supported

## Validation Proccess

In [7]:
# Decidir si reentrenar
if accuracy < 0.95:
    print(f"Retraining model for Wednesday data")

    # Agregar datos del día al conjunto de entrenamiento
    train = train.rbind(valid)

    # Reentrenar modelo
    retrainable_model = h2o.upload_model("models/best_model")

    # Comprobar el tipo de modelo y reentrenar
    if retrainable_model.algo == "DeepLearning":
        dl_checkpoint2 = H2ODeepLearningEstimator(
            model_id="best_model_DL_" + str(retrainable_model.params['epochs']['actual'] + 1),
            checkpoint=retrainable_model.model_id + "_wednesday",
            epochs=int(retrainable_model.params['epochs']['actual']) + 5,
            seed=retrainable_model.params['seed']['actual']
        )
    elif retrainable_model.algo == "drf":
        dl_checkpoint2 = H2ORandomForestEstimator(
            model_id="best_model_forest_" + str(retrainable_model.params['ntrees']['actual'] + 1),
            checkpoint=retrainable_model.model_id + "_wednesday",
            ntrees=int(retrainable_model.params['ntrees']['actual']) + 5,
            seed=retrainable_model.params['seed']['actual']
        )

    # Entrenar con el conjunto de entrenamiento completo
    dl_checkpoint2.train(x=predictors, y=response, training_frame=train)

    # Guardar nuevo mejor modelo
    best_model_path = h2o.download_model(model=dl_checkpoint2, path="models/")


In [1]:
h2o.cluster().shutdown(prompt=False)

NameError: name 'h2o' is not defined